In [1]:
import numpy as np

## Boston Housing Dataset

이번 과제는 보스턴시의 주택 가격과 관련 데이터를 활용하여 보스턴 시의 집값을 예측하는 문제를 푸는 것입니다. Single-layer Neural Network를 활용하여 보스턴의 집값을 예측하는 알고리즘을 구현하세요.

각 컬럼에 대한 설명은 다음과 같습니다. 출처: [ai-times](http://ai-times.tistory.com/431 [ai-times])

  * **CRIM**: 자치 시(town) 별 1인당 범죄율
  * **ZN**: 25,000 평방피트를 초과하는 거주지역의 비율
  * **INDUS**: 비소매상업지역이 점유하고 있는 토지의 비율
  * **CHAS**: 찰스강의 경계에 위치해 있으면 1, 그렇지 않으면 0
  * **NOX**: 10ppm당 농축 일산화질소
  * **RM**: 주택 1가구당 평균 방의 개수
  * **AGE**: 1940년 이전에 건축된 소유주택의 비율
  * **DIS**: 5개의 보스턴 직업센터까지의 접근성 지수
  * **RAD**: 방사형 도로까지의 접근성 지수
  * **TAX**: 10,000 달러 당 재산세율
  * **PTRATIO**: 자치 시(town)별 학생/교사 비율
  * **B**: 1000(Bk-0.63)^2, 여기서 Bk는 자치시별 흑인의 비율을 말함.
  * **LSTAT**: 모집단의 하위계층 비율(%)
  * **MEDV**: 본인 소유의 주택가격(중앙값) (단위: $1,000)
  
** 주의 사항 **
  * **MEDV**가 label(y), 나머지가 feature(X)라고 가정하고 문제를 풀어주세요.
  * **한 번에 너무 잘 풀려는 시도를 하지 마세요.** 처음에는 어떻게든 동작하는 코드를 만드는데 집중하고, 그 다음에 코드를 개성하세요.
  * 현실 데이터는 이전에 우리가 다룬 가상의 데이터와 다르기 때문에, error가 0에 가깝게 내려가지 않을 수도 있습니다. (```error = np.abs(y_predict - y).mean()```) **Boston Housing Dataset에서 error는 5 미만으로 내려가면 충분합니다. ** (=$5,000)
  * error가 수렴하지 않고 발산한다면, **learning_rate를 작게 조정해보세요**.

In [1]:
from sklearn.datasets import load_boston

boston = load_boston()
boston["data"]

array([[6.3200e-03, 1.8000e+01, 2.3100e+00, ..., 1.5300e+01, 3.9690e+02,
        4.9800e+00],
       [2.7310e-02, 0.0000e+00, 7.0700e+00, ..., 1.7800e+01, 3.9690e+02,
        9.1400e+00],
       [2.7290e-02, 0.0000e+00, 7.0700e+00, ..., 1.7800e+01, 3.9283e+02,
        4.0300e+00],
       ...,
       [6.0760e-02, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9690e+02,
        5.6400e+00],
       [1.0959e-01, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9345e+02,
        6.4800e+00],
       [4.7410e-02, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9690e+02,
        7.8800e+00]])

In [3]:
X = boston["data"]

print(X.shape)
X[0]

(506, 13)


array([6.320e-03, 1.800e+01, 2.310e+00, 0.000e+00, 5.380e-01, 6.575e+00,
       6.520e+01, 4.090e+00, 1.000e+00, 2.960e+02, 1.530e+01, 3.969e+02,
       4.980e+00])

In [4]:
y = boston["target"]
print(y.shape)
y[0:10]

(506,)


array([24. , 21.6, 34.7, 33.4, 36.2, 28.7, 22.9, 27.1, 16.5, 18.9])

In [5]:
import pandas as pd

data = pd.DataFrame(X, columns=boston["feature_names"])
data["MEDV"] = y

print(data.shape)
data.head()

(506, 14)


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


### Gradient Descent

In [6]:
x1 = X[:, 0] # CRIM
x2 = X[:, 1] # ZN
x3 = X[:, 2] # INDUS
x4 = X[:, 3] # CHAS
x5 = X[:, 4] # NOX
x6 = X[:, 5] # RM
x7 = X[:, 6] # AGE
x8 = X[:, 7] # DIS
x9 = X[:, 8] # RAD
x10 = X[:, 9] # TAX
x11 = X[:, 10] # PTRATIO
x12 = X[:, 11] # B
x13 = X[:, 12] # LSTAT

In [7]:
num_epoch = 100000
learning_rate = 0.000003

w1 = np.random.uniform(low=-1.0, high=1.0)
w2 = np.random.uniform(low=-1.0, high=1.0)
w3 = np.random.uniform(low=-1.0, high=1.0)
w4 = np.random.uniform(low=-1.0, high=1.0)
w5 = np.random.uniform(low=-1.0, high=1.0)
w6 = np.random.uniform(low=-1.0, high=1.0)
w7 = np.random.uniform(low=-1.0, high=1.0)
w8 = np.random.uniform(low=-1.0, high=1.0)
w9 = np.random.uniform(low=-1.0, high=1.0)
w10 = np.random.uniform(low=-1.0, high=1.0)
w11 = np.random.uniform(low=-1.0, high=1.0)
w12 = np.random.uniform(low=-1.0, high=1.0)
w13 = np.random.uniform(low=-1.0, high=1.0)
b = np.random.uniform(low=-1.0, high=1.0)

for epoch in range(num_epoch):
    y_predict = x1 * w1 + \
                x2 * w2 + \
                x3 * w3 + \
                x4 * w4 + \
                x5 * w5 + \
                x6 * w6 + \
                x7 * w7 + \
                x8 * w8 + \
                x9 * w9 + \
                x10 * w10 + \
                x11 * w11 + \
                x12 * w12 + \
                x13 * w13 + \
                b

    error = np.abs(y_predict - y).mean()
    if error < 5:
        break

    if epoch % 10000 == 0:
        print("{0:5} error = {1:.5f}".format(epoch, error))

    w1 = w1 - learning_rate * ((y_predict - y) * x1).mean()
    w2 = w2 - learning_rate * ((y_predict - y) * x2).mean()
    w3 = w3 - learning_rate * ((y_predict - y) * x3).mean()
    w4 = w4 - learning_rate * ((y_predict - y) * x4).mean()
    w5 = w5 - learning_rate * ((y_predict - y) * x5).mean()
    w6 = w6 - learning_rate * ((y_predict - y) * x6).mean()
    w7 = w7 - learning_rate * ((y_predict - y) * x7).mean()
    w8 = w8 - learning_rate * ((y_predict - y) * x8).mean()
    w9 = w9 - learning_rate * ((y_predict - y) * x9).mean()
    w10 = w10 - learning_rate * ((y_predict - y) * x10).mean()
    w11 = w11 - learning_rate * ((y_predict - y) * x11).mean()
    w12 = w12 - learning_rate * ((y_predict - y) * x12).mean()
    w13 = w13 - learning_rate * ((y_predict - y) * x13).mean()
    
    b = b - learning_rate * (y_predict - y).mean()
    
print("----" * 10)
print("{0:5} error = {1:.5f}".format(epoch, error))

    0 error = 142.27694
10000 error = 6.22973
20000 error = 5.84910
30000 error = 5.66764
40000 error = 5.55011
50000 error = 5.46007
60000 error = 5.38302
70000 error = 5.31489
80000 error = 5.25282
90000 error = 5.19432
----------------------------------------
99999 error = 5.13871


In [8]:
y_predict = x1 * w1 + \
            x2 * w2 + \
            x3 * w3 + \
            x4 * w4 + \
            x5 * w5 + \
            x6 * w6 + \
            x7 * w7 + \
            x8 * w8 + \
            x9 * w9 + \
            x10 * w10 + \
            x11 * w11 + \
            x12 * w12 + \
            x13 * w13 + \
            b
            
result = data.copy()
result["MEDV(predict)"] = y_predict

print(result.shape)
result.head()

(506, 15)


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV,MEDV(predict)
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0,28.079054
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6,25.854274
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7,28.575691
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4,28.709091
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2,27.628170


### Gradient Descent (use dot product)

만일 위의 과제가 너무 쉽다는 생각이 들면, **matrix의 [dot product](https://mathinsight.org/dot_product_matrix_notation)를 활용하여 문제를 풀어보세요.**

In [9]:
num_epoch = 100000
learning_rate = 0.000003

num_data = X.shape[0]

w = np.random.uniform(low=-1.0, high=1.0, size=13)
b = np.random.uniform(low=-1.0, high=1.0)

for epoch in range(num_epoch):
    y_predict = X.dot(w) + b

    error = np.abs(y_predict - y).mean()
    if error < 5:
        break

    if epoch % 10000 == 0:
        print("{0:5} error = {1:.5f}".format(epoch, error))

    w = w - learning_rate * (y_predict - y).dot(X) / num_data
    b = b - learning_rate * (y_predict - y).mean()
    
print("----" * 10)
print("{0:5} error = {1:.5f}".format(epoch, error))

    0 error = 31.44556
10000 error = 6.39985
20000 error = 5.80554
30000 error = 5.51091
40000 error = 5.30311
50000 error = 5.15130
60000 error = 5.03350
----------------------------------------
63271 error = 4.99999


In [10]:
y_predict = X.dot(w) + b

result = data.copy()
result["MEDV(predict)"] = y_predict

print(result.shape)
result.head()

(506, 15)


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV,MEDV(predict)
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0,29.572716
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6,25.278951
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7,27.882493
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4,27.640496
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2,26.850238
